In [1]:

import h2o
from h2o.automl import H2OAutoML
from h2o.estimators.pca import H2OPrincipalComponentAnalysisEstimator
from h2o.transforms.decomposition import H2OPCA
import tempfile
import os
from os import listdir
from os import path
from os.path import isfile, join, getsize
import pandas as pd
import numpy as np
from pycaret.classification import *
import seaborn as sns
import pickle
import tqdm


Bad key "text.kerning_factor" on line 4 in
/opt/tljh/user/lib/python3.7/site-packages/matplotlib/mpl-data/stylelib/_classic_test_patch.mplstyle.
You probably need to get an updated matplotlibrc file from
https://github.com/matplotlib/matplotlib/blob/v3.1.3/matplotlibrc.template
or from the matplotlib source distribution


In [2]:
def apk(actual, predicted, k=10):
    """
    Computes the average precision at k.
    This function computes the average prescision at k between two lists of
    items.
    Parameters
    ----------
    actual : list
             A list of elements that are to be predicted (order doesn't matter)
    predicted : list
                A list of predicted elements (order does matter)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The average precision at k over the input lists
    """
    if len(predicted)>k:
        predicted = predicted[:k]

    score = 0.0
    num_hits = 0.0

    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)

    if not actual:
        return 0.0

    return score / min(len(actual), k)

def mapk(actual, predicted, k=10):
    """
    Computes the mean average precision at k.
    This function computes the mean average prescision at k between two lists
    of lists of items.
    Parameters
    ----------
    actual : list
             A list of lists of elements that are to be predicted 
             (order doesn't matter in the lists)
    predicted : list
                A list of lists of predicted elements
                (order matters in the lists)
    k : int, optional
        The maximum number of predicted elements
    Returns
    -------
    score : double
            The mean average precision at k over the input lists
    """
    return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])



In [3]:
def compute_ap(recall, precision):
    """ Compute the average precision, given the recall and precision curves.
    Code originally from https://github.com/rbgirshick/py-faster-rcnn.
    # Arguments
        recall:    The recall curve (list).
        precision: The precision curve (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """
    # correct AP calculation
    # first append sentinel values at the end
    mrec = np.concatenate(([0.0], recall, [1.0]))
    mpre = np.concatenate(([0.0], precision, [0.0]))

    # compute the precision envelope
    for i in range(mpre.size - 1, 0, -1):
        mpre[i - 1] = np.maximum(mpre[i - 1], mpre[i])

    # to calculate area under PR curve, look for points
    # where X axis (recall) changes value
    i = np.where(mrec[1:] != mrec[:-1])[0]

    # and sum (\Delta recall) * prec
    ap = np.sum((mrec[i + 1] - mrec[i]) * mpre[i + 1])
    return ap

def ap_per_class(tp, conf, pred_cls, target_cls):
    """ Compute the average precision, given the recall and precision curves.
    Source: https://github.com/rafaelpadilla/Object-Detection-Metrics.
    # Arguments
        tp:    True positives (list).
        conf:  Objectness value from 0-1 (list).
        pred_cls: Predicted object classes (list).
        target_cls: True object classes (list).
    # Returns
        The average precision as computed in py-faster-rcnn.
    """

    # Sort by objectness
    i = np.argsort(-conf)
    tp, conf, pred_cls = tp[i], conf[i], pred_cls[i]

    # Find unique classes
    unique_classes = np.unique(target_cls)

    # Create Precision-Recall curve and compute AP for each class
    ap, p, r = [], [], []
    for c in tqdm.tqdm(unique_classes, desc="Computing AP"):
        i = pred_cls == c
        n_gt = (target_cls == c).sum()  # Number of ground truth objects
        n_p = i.sum()  # Number of predicted objects

        if n_p == 0 and n_gt == 0:
            continue
        elif n_p == 0 or n_gt == 0:
            ap.append(0)
            r.append(0)
            p.append(0)
        else:
            # Accumulate FPs and TPs
            fpc = (1 - tp[i]).cumsum()
            tpc = (tp[i]).cumsum()

            # Recall
            recall_curve = tpc / (n_gt + 1e-16)
            r.append(recall_curve[-1])

            # Precision
            precision_curve = tpc / (tpc + fpc)
            p.append(precision_curve[-1])

            # AP from recall-precision curve
            ap.append(compute_ap(recall_curve, precision_curve))

    # Compute F1 score (harmonic mean of precision and recall)
    p, r, ap = np.array(p), np.array(r), np.array(ap)
    f1 = 2 * p * r / (p + r + 1e-16)

    return p, r, ap, f1, unique_classes.astype("int32")

In [4]:
h2o.init(max_mem_size="64G", nthreads = 30)
#nthreads = 30

Checking whether there is an H2O instance running at http://localhost:54321 ..... not found.
Attempting to start a local H2O server...
  Java Version: openjdk version "11.0.7" 2020-04-14; OpenJDK Runtime Environment (build 11.0.7+10-post-Ubuntu-2ubuntu218.04); OpenJDK 64-Bit Server VM (build 11.0.7+10-post-Ubuntu-2ubuntu218.04, mixed mode, sharing)
  Starting server from /opt/tljh/user/lib/python3.7/site-packages/h2o/backend/bin/h2o.jar
  Ice root: /tmp/tmpsxu2wbg_
  JVM stdout: /tmp/tmpsxu2wbg_/h2o_jupyter_chandler_vaughn_started_from_python.out
  JVM stderr: /tmp/tmpsxu2wbg_/h2o_jupyter_chandler_vaughn_started_from_python.err
  Server is running at http://127.0.0.1:54321
Connecting to H2O server at http://127.0.0.1:54321 ... successful.


H2O_cluster_uptime:,01 secs
H2O_cluster_timezone:,Etc/UTC
H2O_data_parsing_timezone:,UTC
H2O_cluster_version:,3.30.0.3
H2O_cluster_version_age:,1 month and 6 days
H2O_cluster_name:,H2O_from_python_jupyter_chandler_vaughn_fwvlq1
H2O_cluster_total_nodes:,1
H2O_cluster_free_memory:,64 Gb
H2O_cluster_total_cores:,32
H2O_cluster_allowed_cores:,30
H2O_cluster_status:,"accepting new members, healthy"


In [5]:
os.chdir('/home/jupyter-chandler.vaughn/data/')
infile = open('merged_training.pickle','rb')
merge_pairs_plus_random = pickle.load(infile)
infile.close()
merge_pairs_plus_random = merge_pairs_plus_random.drop(columns=['pair', 'song_1', 'song_2', 'pair_index', 'sim_distance', 'sim_distance_argmin'])
merge_pairs_plus_random.reset_index(drop=True, inplace=True)

from sklearn.decomposition import PCA
x = merge_pairs_plus_random.drop(['target'], axis=1)
pca = PCA(n_components=4)
principalComponents = pca.fit_transform(x)
principalDf = pd.DataFrame(data = principalComponents
             , columns = ['PC1', 'PC2', 'PC3', 'PC4'])
finalDf = pd.concat([principalDf, merge_pairs_plus_random[['target']]], axis = 1)

# create an export checkpoints directory
checkpoints_dir = tempfile.mkdtemp()

dataset = h2o.H2OFrame(finalDf)
#dataset = h2o.import_file('final_dataset_new.csv')
dataset.shape

Parse progress: |█████████████████████████████████████████████████████████| 100%


(314, 5)

In [6]:
# Identify predictors and response
x = dataset.columns
y = "target"
x.remove(y)

#dataset[y].unique()
dataset[y] = dataset[y].asfactor()

In [7]:
#impute any na's to zero
dataset[x].impute(column = 0, values = [0 for c in range(dataset[x].ncol)])
dataset[x].isna().any()

False

In [8]:
#subset the data so that we can work with it
#eventually we need to do all data for modeling
#using splitframe so we get random rows
#subset_dataset_train, subset_dataset_test, therest = dataset.split_frame(ratios=[0.04, 0.04])

In [9]:
print("Rows To Process: " + str(dataset.nrows))
print("Dataframe Shape: " + str(dataset.shape))

Rows To Process: 314
Dataframe Shape: (314, 5)


In [10]:
#subset the data so that we can work with it
#eventually we need to do all data for modeling
#subset_dataset = dataset[range(0,2000,1),:]
dataset[y] = dataset[y].asfactor()
dataset[x] = dataset[x].asnumeric() 
train,test,valid = dataset.split_frame(ratios=[.7, .15])

In [34]:
aml = H2OAutoML(max_models=10, seed=1, balance_classes = False, exclude_algos = ["GBM"], stopping_metric='misclassification')
aml.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [14]:
# View the AutoML Leaderboard
lb = aml.leaderboard
lb.head(rows=lb.nrows) 

model_id,auc,logloss,aucpr,mean_per_class_error,rmse,mse
GLM_1_AutoML_20200620_012436,0.870456,0.34491,0.808553,0.188392,0.316834,0.100384
DeepLearning_grid__2_AutoML_20200620_012436_model_1,0.869765,0.885135,0.798331,0.197538,0.450378,0.20284
DeepLearning_grid__1_AutoML_20200620_012436_model_22,0.869535,0.567229,0.79982,0.206972,0.363617,0.132217
DeepLearning_grid__1_AutoML_20200620_012436_model_10,0.865739,0.353637,0.777881,0.210021,0.332601,0.110623
DeepLearning_grid__1_AutoML_20200620_012436_model_14,0.865624,0.406459,0.780929,0.17752,0.342314,0.117179
DeepLearning_grid__1_AutoML_20200620_012436_model_15,0.865509,0.91478,0.800164,0.198113,0.428012,0.183194
DeepLearning_grid__1_AutoML_20200620_012436_model_4,0.865163,0.356328,0.79535,0.197538,0.325815,0.106156
DeepLearning_grid__1_AutoML_20200620_012436_model_27,0.864703,0.393428,0.796544,0.20726,0.345964,0.119691
DeepLearning_grid__3_AutoML_20200620_012436_model_11,0.864013,0.382537,0.800052,0.198113,0.341838,0.116854
DeepLearning_grid__1_AutoML_20200620_012436_model_17,0.864013,0.356522,0.803827,0.194777,0.322612,0.104079


In [25]:
# Get model ids for all models in the AutoML Leaderboard
#model_ids = list(aml.leaderboard['model_id'].as_data_frame().iloc[:,0])
# Get the "All Models" Stacked Ensemble model
se = h2o.get_model('DeepLearning_grid__2_AutoML_20200620_012436_model_1')



In [26]:
se

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_grid__2_AutoML_20200620_012436_model_1


Status of Neuron Layers: predicting target, 2-class classification, bernoulli distribution, CrossEntropy loss, 41,602 weights/biases, 495.8 KB, 1,279,810 training samples, mini-batch size 1


,,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
0,,1,4,Input,20,,,,,,,,,
1,,2,200,RectifierDropout,30,0,0,0.287931,0.372088,0,-0.0749452,1.16137,-0.653209,0.727584
2,,3,200,RectifierDropout,30,0,0,0.270729,0.363015,0,-0.104512,0.274908,-0.177344,0.808933
3,,4,2,Softmax,,0,0,0.219364,0.319022,0,-0.598771,0.989431,-20.2853,2.0278




ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.15433401426602578
RMSE: 0.39285368047916486
LogLoss: 0.5523529557268314
Mean Per-Class Error: 0.024334811529933553
AUC: 0.9973392461197339
AUCPR: 0.9973367074641388
Gini: 0.9946784922394678

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9936606205773726: 


,,0,1,Error,Rate
0,0,159.0,5.0,0.0305,(5.0/164.0)
1,1,3.0,162.0,0.0182,(3.0/165.0)
2,Total,162.0,167.0,0.0243,(8.0/329.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.8987315,0.037827823,0.9318182,0.84090906,0.9302326,0.88372093,0.90697676
1,auc,0.8754718,0.073813446,0.9614325,0.8236915,0.93465906,0.7848485,0.8727273
2,aucpr,0.8062657,0.09378886,0.9188738,0.7136475,0.87803864,0.71260816,0.8081604
3,err,0.1012685,0.037827823,0.06818182,0.1590909,0.069767445,0.11627907,0.093023255
4,err_count,4.4,1.67332,3.0,7.0,3.0,5.0,4.0
5,f0point5,0.8361893,0.09882452,0.9302326,0.6976744,0.9302326,0.7894737,0.8333333
6,f1,0.7598899,0.09113071,0.84210527,0.6315789,0.84210527,0.7058824,0.7777778
7,f2,0.69656986,0.08563402,0.7692308,0.5769231,0.7692308,0.63829786,0.7291667
8,lift_top_group,4.101818,0.184682,4.0,4.0,3.909091,4.3,4.3
9,logloss,0.88607883,0.26353973,0.621051,0.9463421,0.7219727,0.8370857,1.3039427



See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2020-06-20 01:47:16,0.000 sec,None,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2020-06-20 01:47:16,30.677 sec,40617 obs/sec,10.0,1,3290.0,0.705522,4.204744,-0.991065,0.831135,0.870195,1.993939,0.224924
2,,2020-06-20 01:47:21,35.708 sec,87396 obs/sec,1350.0,135,444150.0,0.500001,1.204362,-0.000015,0.972395,0.972417,1.993939,0.085106
3,,2020-06-20 01:47:26,40.728 sec,99439 obs/sec,3040.0,304,1000160.0,0.495404,1.332034,0.018289,0.988322,0.988519,1.993939,0.063830
4,,2020-06-20 01:47:29,43.318 sec,101387 obs/sec,3890.0,389,1279810.0,0.392854,0.552353,0.382658,0.997339,0.997337,1.993939,0.024316



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,PC1,1.000000,1.000000,0.414133
1,PC3,0.504137,0.504137,0.208779
2,PC2,0.475346,0.475346,0.196856
3,PC4,0.435203,0.435203,0.180232


In [27]:
pred = se.predict(test)
pred

deeplearning prediction progress: |███████████████████████████████████████| 100%


predict,p0,p1
1,9.73981e-66,1
1,3.38434e-103,1
1,1.08013e-20,1
1,1.0723e-130,1
1,8.00722e-151,1
0,1,9.18072e-18
0,0.999608,0.0003925
0,0.992045,0.00795522
0,0.999972,2.7575e-05
0,0.273473,0.726527


In [28]:
results = pd.concat([test.as_data_frame(), pred.as_data_frame()], axis=1)
results['rank'] = results['p1'].rank(ascending=False)

In [29]:
#results_sorted = results.sort_values(by=['p1'], ascending=False)
results_sorted = results.sort_values(by=['rank'], ascending=True)
results_sorted

,PC1,PC2,PC3,PC4,target,predict,p0,p1,rank
0,1.264385,-0.268004,0.066505,-0.273680,1,1,9.739813e-66,1.000000e+00,4.0
1,1.625947,-0.085088,0.102404,0.058911,1,1,3.384336e-103,1.000000e+00,4.0
2,0.811489,0.601796,-0.399470,-0.514232,1,1,1.080131e-20,1.000000e+00,4.0
3,1.983505,0.237813,0.112501,-0.243212,1,1,1.072295e-130,1.000000e+00,4.0
4,2.261934,0.371216,-0.080076,0.009554,1,1,8.007218e-151,1.000000e+00,4.0
39,0.715782,-0.214356,-0.023059,0.149530,0,1,2.136053e-17,1.000000e+00,4.0
10,1.017949,-0.659255,-0.036226,-0.091441,1,1,3.983030e-53,1.000000e+00,4.0
23,0.160795,-0.046641,-0.064023,-0.174417,0,1,6.527500e-06,9.999935e-01,8.0
38,0.610192,-0.286250,-0.287269,0.164064,0,1,1.392562e-05,9.999861e-01,9.0
29,0.480966,0.066658,0.289478,0.102364,0,1,1.607519e-04,9.998392e-01,10.0


In [30]:
#find Average Precision
tp = results_sorted['target'].values
conf = results_sorted['p1'].values
pred_cls = results_sorted['predict'].values
target_cls = [results_sorted['target'].values]

p, r, ap, f1, unique_classes = ap_per_class(tp, conf, pred_cls, target_cls)

Computing AP: 100%|██████████| 2/2 [00:00<00:00, 564.43it/s]


In [31]:
#MAP
np.mean(ap)

0.27784715284715283

In [33]:
perf = se.model_performance(valid)
perf


ModelMetricsBinomial: deeplearning
** Reported on test data. **

MSE: 0.37398327668485243
RMSE: 0.6115417211318067
LogLoss: 2.545835186866052
Mean Per-Class Error: 0.3325791855203619
AUC: 0.7160633484162896
AUCPR: 0.5067015644758425
Gini: 0.4321266968325792

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.003083013019186579: 


,,0,1,Error,Rate
0,0,14.0,20.0,0.5882,(20.0/34.0)
1,1,1.0,12.0,0.0769,(1.0/13.0)
2,Total,15.0,32.0,0.4468,(21.0/47.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,3.083013e-03,0.533333,28.0
1,max f2,1.130907e-05,0.738636,32.0
2,max f0point5,1.000000e+00,0.606061,1.0
3,max accuracy,1.000000e+00,0.787234,1.0
4,max precision,1.000000e+00,0.800000,1.0
5,max recall,1.130907e-05,1.000000,32.0
6,max specificity,1.000000e+00,0.970588,0.0
7,max absolute_mcc,1.000000e+00,0.403724,1.0
8,max min_per_class_accuracy,8.846163e-01,0.617647,18.0
9,max mean_per_class_accuracy,3.083013e-03,0.667421,28.0



Gains/Lift Table: Avg response rate: 27.66 %, avg score: 51.76 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.085106,1.000000e+00,2.711538,2.711538,0.75,1.000000e+00,0.750000,1.000000,0.230769,0.230769,171.153846,171.153846
1,,2,0.106383,9.999998e-01,3.615385,2.892308,1.00,1.000000e+00,0.800000,1.000000,0.076923,0.307692,261.538462,189.230769
2,,3,0.148936,9.960384e-01,0.000000,2.065934,0.00,9.986360e-01,0.571429,0.999610,0.000000,0.307692,-100.000000,106.593407
3,,4,0.212766,9.920266e-01,0.000000,1.446154,0.00,9.947240e-01,0.400000,0.998144,0.000000,0.307692,-100.000000,44.615385
4,,5,0.297872,9.697764e-01,1.807692,1.549451,0.50,9.817197e-01,0.428571,0.993452,0.153846,0.461538,80.769231,54.945055
5,,6,0.404255,9.287015e-01,1.446154,1.522267,0.40,9.602359e-01,0.421053,0.984711,0.153846,0.615385,44.615385,52.226721
6,,7,0.510638,7.334339e-01,0.723077,1.355769,0.20,8.521916e-01,0.375000,0.957103,0.076923,0.692308,-27.692308,35.576923
7,,8,0.595745,8.265378e-02,0.903846,1.291209,0.25,3.076208e-01,0.357143,0.864319,0.076923,0.769231,-9.615385,29.120879
8,,9,0.702128,5.143790e-04,1.446154,1.314685,0.40,2.555280e-02,0.363636,0.737234,0.153846,0.923077,44.615385,31.468531
9,,10,0.787234,3.032721e-07,0.903846,1.270270,0.25,3.290379e-05,0.351351,0.657536,0.076923,1.000000,-9.615385,27.027027


In [129]:
#Lets look at Plagerized data
plagerize_unseen = pd.read_pickle('merged_unseen.pickle')

In [130]:
plagerize_unseen['sim_distance_bass'] = np.log(plagerize_unseen['sim_distance_bass']+0.0001)
plagerize_unseen['sim_distance_drums'] = np.log(plagerize_unseen['sim_distance_drums']+0.0001)
plagerize_unseen['sim_distance_vocals'] = np.log(plagerize_unseen['sim_distance_vocals']+0.0001)
plagerize_unseen['sim_distance_other'] = np.log(plagerize_unseen['sim_distance_other']+0.0001)
plagerize_unseen['sim_distance_argmin_combined'] = plagerize_unseen[['sim_distance','sim_distance_argmin']].min(axis=1)
plagerize_unseen['sim_distance_argmin_combined'] = np.log(plagerize_unseen['sim_distance_argmin_combined']+0.0001)

In [131]:
plagerize_unseen = plagerize_unseen.drop(columns=['pair_x', 'song_1', 'song_2', 'pair_y', 'sim_distance', 'sim_distance_argmin'])
plagerize_unseen.reset_index(drop=True, inplace=True)
plagerize_unseen = h2o.H2OFrame(plagerize_unseen)

Parse progress: |█████████████████████████████████████████████████████████| 100%


In [132]:
pred_unseen = aml.predict(plagerize_unseen)

stackedensemble prediction progress: |████████████████████████████████████| 100%


In [133]:
pred_unseen

predict,p0,p1
0,0.645903,0.354097
0,0.891783,0.108217
1,0.612703,0.387297
1,0.565323,0.434677
1,0.404694,0.595306
0,0.62433,0.37567
0,0.834967,0.165033
0,0.756806,0.243194
1,0.14575,0.85425
1,0.157077,0.842923


In [134]:
results_unseen = pd.concat([plagerize_unseen.as_data_frame(), pred_unseen.as_data_frame()], axis=1)
results_unseen['rank'] = results_unseen['p1'].rank(ascending=False)

results_unseen = results_unseen.sort_values(by=['rank'], ascending=True)
results_unseen

,sim_distance_bass,sim_distance_drums,sim_distance_vocals,sim_distance_other,sim_distance_argmin_combined,predict,p0,p1,rank
13,-3.377752,-3.296034,-3.686204,-3.564177,-3.711500,1,0.115353,0.884647,1.0
8,-3.176397,-3.252703,-2.836602,-3.490970,-3.490970,1,0.145750,0.854250,2.0
9,-3.880038,-2.531039,-2.979819,-3.406609,-3.880038,1,0.157077,0.842923,3.0
4,-1.731434,-3.445135,-3.127537,-3.144894,-3.445135,1,0.404694,0.595306,4.0
15,-3.130439,-3.375734,-2.162241,-3.175864,-3.529704,1,0.414288,0.585712,5.0
14,-3.180531,-3.027164,-2.616983,-3.170255,-3.276949,1,0.441112,0.558888,6.0
20,-2.514196,-3.412679,-2.862315,-3.117153,-3.412679,1,0.536015,0.463985,7.0
25,-3.110523,-3.140508,-3.007707,-3.255698,-3.287768,1,0.563437,0.436563,8.0
3,-2.921536,-3.036883,-2.442322,-3.098474,-3.098474,1,0.565323,0.434677,9.0
2,-3.021577,-3.274226,-3.318774,-2.753680,-3.318774,1,0.612703,0.387297,10.0


In [157]:
aml2 = H2OAutoML(max_models=100, seed=1, balance_classes = True, exclude_algos = ["GBM"], stopping_metric='logloss')
aml2.train(x=x, y=y, training_frame=train)

AutoML progress: |████████████████████████████████████████████████████████| 100%


In [13]:
# View the AutoML Leaderboard
lb2 = aml2.leaderboard
lb2.head(rows=lb2.nrows) 

NameError: name 'aml2' is not defined

In [159]:
aml2.leader

Model Details
H2ODeepLearningEstimator :  Deep Learning
Model Key:  DeepLearning_grid__1_AutoML_20200618_010541_model_6


Status of Neuron Layers: predicting target, 2-class classification, bernoulli distribution, CrossEntropy loss, 4,002 weights/biases, 56.1 KB, 1,323,840 training samples, mini-batch size 1


,,layer,units,type,dropout,l1,l2,mean_rate,rate_rms,momentum,mean_weight,weight_rms,mean_bias,bias_rms
0,,1,5,Input,5,,,,,,,,,
1,,2,500,RectifierDropout,30,0,0,0.0948417,0.190474,0,0.130583,0.648805,-0.4535,0.454669
2,,3,2,Softmax,,0,0,0.116843,0.203498,0,-0.287446,1.7959,-13.9339,0.856533




ModelMetricsBinomial: deeplearning
** Reported on train data. **

MSE: 0.003606874034523682
RMSE: 0.06005725630199636
LogLoss: 0.014922674337819183
Mean Per-Class Error: 0.0
AUC: 1.0
AUCPR: 1.0
Gini: 1.0

Confusion Matrix (Act/Pred) for max f1 @ threshold = 0.9649643070089776: 


,,0,1,Error,Rate
0,0,168.0,0.0,0.0,(0.0/168.0)
1,1,0.0,168.0,0.0,(0.0/168.0)
2,Total,168.0,168.0,0.0,(0.0/336.0)



Maximum Metrics: Maximum metrics at their respective thresholds


...

,,mean,sd,cv_1_valid,cv_2_valid,cv_3_valid,cv_4_valid,cv_5_valid
0,accuracy,0.9209302,0.072802216,1.0,0.88372093,0.81395346,0.95348835,0.95348835
1,auc,0.92717767,0.08168107,1.0,0.8090909,0.875817,0.9836601,0.96732026
2,aucpr,0.86196584,0.13269758,1.0,0.72241664,0.71788657,0.95133847,0.9181876
3,err,0.07906977,0.072802216,0.0,0.11627907,0.18604651,0.046511628,0.046511628
4,err_count,3.4,3.130495,0.0,5.0,8.0,2.0,2.0
5,f0point5,0.8293923,0.15820979,1.0,0.7894737,0.5797101,0.8888889,0.8888889
6,f1,0.83006537,0.13956966,1.0,0.7058824,0.6666667,0.8888889,0.8888889
7,f2,0.8400779,0.13616619,1.0,0.63829786,0.78431374,0.8888889,0.8888889
8,lift_top_group,4.5866666,0.26168966,4.3,4.3,4.7777777,4.7777777,4.7777777
9,logloss,1.4757744,0.77236855,1.1861598,2.56315,1.8991042,1.1776481,0.5528102



See the whole table with table.as_data_frame()

Scoring History: 


,,timestamp,duration,training_speed,epochs,iterations,samples,training_rmse,training_logloss,training_r2,training_auc,training_pr_auc,training_lift,training_classification_error
0,,2020-06-18 01:09:47,0.000 sec,None,0.0,0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,,2020-06-18 01:09:47,3 min 52.485 sec,45405 obs/sec,10.0,1,3360.0,0.683058,4.984687,-0.866272,0.890660,0.881439,2.0,0.226190
2,,2020-06-18 01:09:52,3 min 57.509 sec,100813 obs/sec,1520.0,152,510720.0,0.300829,0.385952,0.638007,0.997732,0.997631,2.0,0.017857
3,,2020-06-18 01:09:57,4 min 2.518 sec,116104 obs/sec,3470.0,347,1165920.0,0.177339,0.096239,0.874204,0.999858,0.999857,2.0,0.002976
4,,2020-06-18 01:09:58,4 min 3.700 sec,118422 obs/sec,3940.0,394,1323840.0,0.060057,0.014923,0.985573,1.000000,1.000000,2.0,0.000000



Variable Importances: 


,variable,relative_importance,scaled_importance,percentage
0,sim_distance_other,1.000000,1.000000,0.278376
1,sim_distance_argmin_combined,0.829031,0.829031,0.230782
2,sim_distance_vocals,0.754230,0.754230,0.209959
3,sim_distance_bass,0.532916,0.532916,0.148351
4,sim_distance_drums,0.476091,0.476091,0.132532


In [160]:
pred2 = aml2.predict(test)
pred2

deeplearning prediction progress: |███████████████████████████████████████| 100%


predict,p0,p1
1,3.38781e-50,1
1,4.89301e-109,1
1,1.65899e-37,1
1,0.0059793,0.994021
0,1,1.10701e-63
0,1,1.13775e-34
1,5.88561e-172,1
0,0.99669,0.00330997
0,0.999994,6.28546e-06
0,1,2.36223e-19


In [161]:
results2 = pd.concat([test.as_data_frame(), pred2.as_data_frame()], axis=1)
results2['rank'] = results2['p1'].rank(ascending=False)

In [162]:
results2_sorted = results2.sort_values(by=['rank'], ascending=True)
results2_sorted

,sim_distance_bass,sim_distance_drums,sim_distance_vocals,sim_distance_other,target,sim_distance_argmin_combined,predict,p0,p1,rank
0,-3.396255,-3.318984,-3.726317,-3.307305,1,-3.752329,1,3.387806e-50,1.000000e+00,3.5
11,-2.226876,-3.649815,-3.593604,-3.713786,1,-3.862291,1,8.853730e-65,1.000000e+00,3.5
6,-3.748001,-3.524138,-4.008976,-4.432945,1,-4.432945,1,5.885614e-172,1.000000e+00,3.5
14,-3.601015,-3.368839,-3.177789,-3.862225,1,-4.050557,1,1.759876e-74,1.000000e+00,3.5
2,-3.217733,-2.925328,-2.535029,-3.505658,1,-3.812027,1,1.658990e-37,1.000000e+00,3.5
1,-3.793328,-3.451294,-3.797983,-3.908221,1,-4.110113,1,4.893007e-109,1.000000e+00,3.5
25,-2.437577,-2.955856,-2.886427,-3.238617,0,-3.365747,1,9.909716e-05,9.999009e-01,7.0
15,-2.782669,-3.015499,-2.328491,-2.815056,0,-3.015499,1,4.013327e-04,9.995987e-01,8.0
37,-2.994229,-3.466758,-3.234972,-3.114258,0,-3.466758,1,2.885612e-03,9.971144e-01,9.0
3,-2.739924,-3.098317,-3.356697,-3.304992,1,-3.356697,1,5.979300e-03,9.940207e-01,10.0


In [163]:
#find Average Precision
tp = results2_sorted['target'].values
conf = results2_sorted['p1'].values
pred_cls = results2_sorted['predict'].values
target_cls = [results2_sorted['target'].values]

p, r, ap, f1, unique_classes = ap_per_class(tp, conf, pred_cls, target_cls)

Computing AP: 100%|██████████| 2/2 [00:00<00:00, 1278.36it/s]


In [164]:
#MAP
np.mean(ap)

0.2719764142287931

In [165]:
perf2 = aml2.leader.model_performance(valid)
perf2


ModelMetricsBinomial: deeplearning
** Reported on test data. **

MSE: 0.27584510106617555
RMSE: 0.5252095782315623
LogLoss: 3.128556380625003
Mean Per-Class Error: 0.22857142857142865
AUC: 0.7904761904761906
AUCPR: 0.6650018748993816
Gini: 0.5809523809523811

Confusion Matrix (Act/Pred) for max f1 @ threshold = 4.00081380260415e-08: 


,,0,1,Error,Rate
0,0,19.0,16.0,0.4571,(16.0/35.0)
1,1,0.0,15.0,0.0,(0.0/15.0)
2,Total,19.0,31.0,0.32,(16.0/50.0)



Maximum Metrics: Maximum metrics at their respective thresholds


,metric,threshold,value,idx
0,max f1,4.000814e-08,0.652174,26.0
1,max f2,4.000814e-08,0.824176,26.0
2,max f0point5,1.000000e+00,0.714286,0.0
3,max accuracy,1.000000e+00,0.800000,0.0
4,max precision,1.000000e+00,1.000000,0.0
5,max recall,4.000814e-08,1.000000,26.0
6,max specificity,1.000000e+00,1.000000,0.0
7,max absolute_mcc,4.000814e-08,0.512516,26.0
8,max min_per_class_accuracy,2.627828e-03,0.657143,17.0
9,max mean_per_class_accuracy,4.000814e-08,0.771429,26.0



Gains/Lift Table: Avg response rate: 30.00 %, avg score: 30.66 %


,,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,score,cumulative_response_rate,cumulative_score,capture_rate,cumulative_capture_rate,gain,cumulative_gain
0,,1,0.10,1.000000e+00,3.333333,3.333333,1.0,1.000000e+00,1.000000,1.000000,0.333333,0.333333,233.333333,233.333333
1,,2,0.10,1.000000e+00,0.000000,3.333333,0.0,0.000000e+00,1.000000,1.000000,0.000000,0.333333,-100.000000,233.333333
2,,3,0.16,9.999998e-01,0.000000,2.083333,0.0,9.999999e-01,0.625000,1.000000,0.000000,0.333333,-100.000000,108.333333
3,,4,0.20,9.999960e-01,1.666667,2.000000,0.5,9.999982e-01,0.600000,1.000000,0.066667,0.400000,66.666667,100.000000
4,,5,0.30,6.023977e-01,1.333333,1.777778,0.4,9.499240e-01,0.533333,0.983308,0.133333,0.533333,33.333333,77.777778
5,,6,0.40,5.393055e-03,0.000000,1.333333,0.0,1.142841e-01,0.400000,0.766052,0.000000,0.533333,-100.000000,33.333333
6,,7,0.50,7.557687e-06,2.666667,1.600000,0.8,2.187024e-03,0.480000,0.613279,0.266667,0.800000,166.666667,60.000000
7,,8,0.60,7.764053e-07,1.333333,1.555556,0.4,4.066950e-06,0.466667,0.511066,0.133333,0.933333,33.333333,55.555556
8,,9,0.70,2.811431e-14,0.666667,1.428571,0.2,1.248289e-08,0.428571,0.438057,0.066667,1.000000,-33.333333,42.857143
9,,10,0.80,2.812880e-31,0.000000,1.250000,0.0,3.724589e-22,0.375000,0.383300,0.000000,1.000000,-100.000000,25.000000


In [ ]:
h2o.cluster().shutdown()